In [33]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import datetime
from joblib import Parallel, delayed
import statsmodels.api as sm

In [34]:
AllData = pd.read_excel('C:/Users/Thinkpad/Desktop/MultiTestData.xlsx',index_col=0)
AllDataReturn = AllData.pct_change().dropna()

def ComputeCorr(df):
    corrList = []
    for col in [i for i in df.columns if i != '沪深300']:
        corrList.append(np.corrcoef(df['沪深300'],df[col])[0,1])
        corr = np.mean(corrList)
    return corr

def ComputeRsquare(df):
    y = df['沪深300']
    X = df[[i for i in df.columns if i != '沪深300']]
    rsquared = sm.OLS(y,X).fit().rsquared
    return rsquared

In [35]:
t1 = datetime.datetime.now()
dateList = AllDataReturn.index.tolist()
saveCorrList = []
win = 252
for dateCnt in range(len(dateList)):
    date = dateList[dateCnt]
    AllDataReturnSub = AllDataReturn[AllDataReturn.index<=date].copy()
    if len(AllDataReturnSub)<win:
        corr = np.nan
    else:
        AllDataReturnSub = AllDataReturnSub.iloc[-win:,:].copy()
        corr = ComputeRsquare(AllDataReturnSub)
    saveCorrList.append(corr)
RollingCorr = pd.DataFrame(saveCorrList,columns=['RollingCorr'],index = dateList)
print(datetime.datetime.now()-t1)

0:00:22.307321


In [ ]:
'''
窗口滚动计算 - 并行计算框架
'''

In [37]:
# 首先生成一个用于循环计算的迭代器
DfList = []
dateList = AllDataReturn.index.tolist()
for dateCnt in range(len(dateList)):
    date = dateList[dateCnt]
    AllDataReturnSub = AllDataReturn[AllDataReturn.index<=date].copy()
    AllDataReturnSub = AllDataReturnSub.iloc[-win:,:].copy()
    DfList.append(AllDataReturnSub)

t1 = datetime.datetime.now()

# 对迭代器中每个元素的处理函数，长度不及滚动窗口则返回np.nan，否则使用计算函数
def ComputeCorr(df):
    corrList = []
    for col in [i for i in df.columns if i != '沪深300']:
        corrList.append(np.corrcoef(df['沪深300'],df[col])[0,1])
        corr = np.mean(corrList)
    return corr
def MultiTask(df):
    df = df.copy()
    if len(df)<252:
        corr = np.nan
    else:
        corr = ComputeCorr(df)
    return corr

saveCorrList = Parallel(n_jobs=mp.cpu_count())(delayed(MultiTask)(group) for group in DfList)

RollingCorr = pd.DataFrame(saveCorrList,columns=['RollingCorr'],index = dateList)
print(datetime.datetime.now()-t1)

0:00:04.347367
